In [47]:
!pip install backtrader
!pip install shioaji
import numpy as np
import pandas as pd
import requests
import shioaji as sj
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from matplotlib.dates import date2num
import backtrader as bt
from google.colab import userdata

# 建立API物件，simulation=True是代表測試帳號
api = sj.Shioaji(simulation=True)

#==== for Colab
shioaji_secret=userdata.get('SHIOAJI_SECRETKEY')
shioaji_apikey=userdata.get('SHIOAJI_APIKEY')

# 登入你的key
accounts = api.login(shioaji_apikey, shioaji_secret)



Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [48]:
def get_daily_kbar(stock_code, start_date, end_date):
    kbars = api.kbars(
        contract=api.Contracts.Stocks[stock_code],
        start=start_date,
        end=end_date,
    )
    # 將 'ts' 列重命名為 'datetime'
    df = pd.DataFrame({**kbars}).rename(columns={'ts': 'datetime', 'Open': 'open', 'Close': 'close', 'High': 'high', 'Low': 'low', 'Volume': 'volume'})

    # 將 'datetime' 轉換為日期時間格式
    df['datetime'] = pd.to_datetime(df['datetime'])

    # 依照日期分組，並計算每組的 open, close, high, low, volume
    daily_kbar = df.resample('D', on='datetime').agg({
        'open': 'first',
        'close': 'last',
        'high': 'max',
        'low': 'min',
        'volume': 'sum'  # 計算每日成交量的總和
    }).dropna()  # 移除空白的天數

    # 計算當日漲跌幅
    daily_kbar['previous_close'] = daily_kbar['close'].shift(1)

    # 計算今天的漲跌幅 (今天的 close - 昨天的 close) / 昨天的 close * 100
    daily_kbar['change_percentage'] = ((daily_kbar['close'] - daily_kbar['previous_close']) / daily_kbar['previous_close'] * 100).round(2)

    # 計算當日震幅
    daily_kbar['amplitude'] = ((daily_kbar['high'] - daily_kbar['low']) / daily_kbar['high'] * 100).round(2)

    # 移除前一天沒有數據的行
    daily_kbar = daily_kbar.dropna(subset=['previous_close'])

    print(daily_kbar)

    # 將結果輸出到 Excel 檔案
    # daily_kbar.to_excel(f'{stock_code}-daily_kbar.xlsx', index=True)  # `index=True` 會將日期作為列索引輸出

    return daily_kbar


In [74]:
start_day ="2021-01-01"
end_day = "2023-12-31"

# goodStockList2020=[4743,6443,6531,4128,3545 ,6547,2609 ,8046 ,3218 ,6492 ,2368 ,2603 ,4961 ,2303 ,2615 ,3661 ,5269 ,4142 ,6415, 3374]
# for(stock_code)in goodStockList2020:
#     stock_code = str(stock_code)
#     get_daily_kbar(stock_code,start_day,end_day)

# get_daily_kbar(f"{stock_code}",start_day,end_day)

df_2609 = get_daily_kbar("1519",start_day,end_day)

              open   close    high     low  volume  previous_close  \
datetime                                                             
2021-01-05   48.95   48.75   49.40   48.70    1247           49.00   
2021-01-06   48.70   46.85   48.95   46.55    3972           48.75   
2021-01-07   46.95   47.80   49.45   46.95    3628           46.85   
2021-01-08   48.25   48.50   48.50   46.80    2400           47.80   
2021-01-11   48.70   48.30   49.30   48.15    2095           48.50   
...            ...     ...     ...     ...     ...             ...   
2023-12-25  326.00  333.50  341.00  326.00   12052          321.00   
2023-12-26  338.50  315.00  340.50  314.00   15301          333.50   
2023-12-27  316.50  316.50  323.50  316.50    8549          315.00   
2023-12-28  325.00  348.00  348.00  325.00   18627          316.50   
2023-12-29  358.00  327.00  360.50  317.00   34502          348.00   

            change_percentage  amplitude  
datetime                                  
202

In [79]:
# sma cross strategy
class DailyMAStrategy(bt.Strategy):
  # 設定參數
  params = (('maperiods',[5,10,20,60]),)

  def __init__(self):
    # 定義移動平均線（5MA, 10MA, 20MA, 60MA）
    self.sma5 = bt.ind.SMA(self.data.close, period=self.params.maperiods[0])
    self.sma10 = bt.ind.SMA(self.data.close, period=self.params.maperiods[1])
    self.sma20 = bt.ind.SMA(self.data.close, period=self.params.maperiods[2])
    self.sma60 = bt.ind.SMA(self.data.close, period=self.params.maperiods[3])


  def next(self):
    # 尾盤的時間點 (假設是當日最後一個時間點)
    if len(self.data) < 2:  # 保證有足夠的前一日數據
      return

    # 買入條件
    if not self.position:
      if (self.data.close[0] > self.sma5[0] and
          self.data.close[0] > self.sma10[0] and
          self.data.close[0] > self.sma20[0] and
          self.data.close[0] > self.sma60[0] and
          self.data.volume[0] > 3* self.data.volume[-1]):

          self.buy(size=3000)
          print(f"Buy at {self.data.close[0]} on {self.data.datetime.date(0)}")

    elif self.position:
      if self.data.close[0] < self.sma60[0]:
        print(f"Sell at {self.data.close[0]} on {self.data.datetime.date(0)}")
        self.sell(size=3000)






In [80]:
cerebro = bt.Cerebro()
cerebro.addstrategy(DailyMAStrategy)
cerebro.broker.setcash(10**6)
data = bt.feeds.PandasData(dataname=df_2609)
cerebro.adddata(data)

# 券商的手續費
cerebro.broker.setcommission(commission=0.1425/100)



# 如果跑到最後，還有股票沒有賣出的話，會以當天收盤價的價格來加總
print("初始資金: %.2f" % cerebro.broker.getvalue())

# 執行策略
cerebro.run()

print("結束資金：%.2f" % cerebro.broker.getvalue())
# # 可視化結果
cerebro.plot()

初始資金: 1000000.00
Buy at 44.3 on 2021-07-13
Sell at 43.85 on 2021-07-14
Buy at 36.95 on 2021-12-15
Sell at 36.15 on 2022-01-07
Buy at 39.6 on 2022-01-11
Sell at 36.45 on 2022-02-24
Buy at 38.7 on 2022-03-03
Sell at 37.0 on 2022-03-08
Buy at 42.05 on 2022-03-14
Sell at 38.9 on 2022-04-25
Buy at 37.0 on 2022-07-14
Sell at 39.0 on 2022-10-20
Buy at 40.5 on 2022-11-21
Sell at 39.95 on 2022-11-23
Buy at 42.7 on 2022-12-02
Sell at 251.0 on 2023-10-27
Buy at 275.0 on 2023-11-24
結束資金：1748341.27


[[<Figure size 640x480 with 4 Axes>]]

In [81]:
print(api.usage())

connections=2 bytes=311860143 limit_bytes=524288000 remaining_bytes=212427857
